In [11]:
# Prepare the data

import jieba

titles = [
    "失戀自救手冊",
    "新聞真假掰｜假訊息ByeBye！",
    "聽鉅子騷，我的私房鬼事",
    "冰角新聞",
    "實用色素——女性向成人音聲",
    "南波囡仔",
    "故弄玄虛",
    "一起說故事",
    "豬探長推理故事集",
    "從前從前",
    "唐陽雞酒屋",
    "偷聽史多利 Talking Story",
    "博音",
    "媽爹講故事",
    "Xuwen music",
    "哇賽心理學",
    "呱呱呱樂園｜ガーガー",
    "第一次聽見 Sonic Secret",
    "艾比麻麻說故事",
    "金剛經講要~中道佛學會"
]

seg_titles = []
for title in titles:
    seg = jieba.cut(title, use_paddle=True)
    seg_titles.append(' '.join(list(seg)))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/12/2ws31bxs6mz7g1vzfnx8wrpm0000gn/T/jieba.cache
Loading model cost 0.344 seconds.
Prefix dict has been built successfully.


In [ ]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=["http://localhost:9200/"],
    http_auth=("admin", "admin")
)

In [ ]:
# create index
index_name = "podcast_titles"
client.indices.create(
    index=index_name,
    body={
        "settings": {
            "analysis": {
                "analyzer": {
                    "whitespace": {
                        "type": "whitespace"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "title": {
                    "type": "text",
                    "analyzer": "whitespace"
                }
            }
        }
    }
)

In [ ]:
# insert data
for seg in seg_titles:
    client.index(index=index_name, body={"title": seg})

In [47]:
# search result
client.search(
    index=index_name,
    body={
        "query": {
            "match": {
                "title": {
                    "query": "新聞"
                }
            }
        }
    }
)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 2.6489527,
  'hits': [{'_index': 'podcast_titles',
    '_id': '1liYeYYBx2gTipBlsFjv',
    '_score': 2.6489527,
    '_source': {'title': '冰角 新聞'}},
   {'_index': 'podcast_titles',
    '_id': '1FiYeYYBx2gTipBlsFjX',
    '_score': 1.5945265,
    '_source': {'title': '新聞 真假 掰 ｜ 假訊息 ByeBye ！'}}]}}

In [ ]:
# we always have the second chance
# client.indices.delete(index="index_name")